# 한빛출판네트워크

In [33]:
import requests
from urllib.parse import quote
import pandas as pd
from bs4 import BeautifulSoup

### 1. 데이터 가져오기

In [34]:
site = 'https://www.hanbit.co.kr'
url = 'https://www.hanbit.co.kr/store/books/new_book_list.html'
req = requests.get(url)
html = req.text
soup = BeautifulSoup(html, 'html.parser')

### 찾으려는 데이터의 태그찾기

In [35]:
lis = soup.select_one('.sub_book_list_area').find_all('li')
len(lis)

20

### 하나의 데이터 선택해서 원하는 정보 추출하기

- 제목, 저자, 역자, 출간일, 페이지, 정가, 판가, 마일리지

In [36]:
li = lis[0]

In [37]:
li = lis[0]
href = li.find('a')['href']
href

'/store/books/look.php?p_code=B6694310162'

In [39]:
# 제목
title = li.select('.info > p')[0].get_text().strip()
title 

'하루 한 장 초등과학 365'

In [40]:
# 저자 
author = li.select('.info > p')[1].get_text().strip()
author 

'치바 가즈요시'

### sub 페이지에서 정보추출

- 역자, 출간일, 페이지, 정가, 판가, 마일리지

In [41]:
sub_url = site + href 
sub_url

'https://www.hanbit.co.kr/store/books/look.php?p_code=B6694310162'

In [42]:
sub_req = requests.get(sub_url)
sub_html = sub_req.text
sub_soup = BeautifulSoup(sub_html, 'html.parser')

In [43]:
# 역자 
sub_soup.select('.info_list > li')

[<li><strong>저자 : </strong><span>치바 가즈요시 </span></li>,
 <li><strong>번역 : </strong><span>허영은 , 유우종 (감수) </span></li>,
 <li><strong>출간 : </strong><span>2021-12-14</span></li>,
 <li><strong>페이지 : </strong><span>416 쪽</span></li>,
 <li><strong>ISBN : </strong><span>9791190846295</span></li>,
 <li><strong>물류코드 :</strong><span>5329</span></li>]

In [44]:
sub_lis = sub_soup.select('.info_list > li')
len(sub_lis)

6

In [66]:
# 역자( dict으로 만들어서 번역 키 값이 있으면~없으면~ )
sub_list = sub_soup.select('.info_list > li')[1].get_text()
# lux = dict(sub_soup.select('.info_list > li'))
# lux
sub_list

'출간 : 2021-12-10'

In [62]:
# 역자
if sub_list[:2] == '번역':
    trans = sub_soup.select('.info_list > li')[1].get_text()[5:]
else: 
    trans = print('.')
trans


'허영은 , 유우종 (감수) '

In [14]:
# 출간일 
date = sub_lis[2].get_text().split(':')[1]
date = int(date.replace('-',''))
date

20211214

In [15]:
# 페이지 
page = sub_lis[3].get_text().split(':')[1]
page = int(page.replace('쪽',''))
page

416

In [16]:
sub_soup.select_one('.payment_box.curr').find_all('p')

[<p><span class="pbl">정가 : </span><span class="pbr"><del>19,800원</del></span></p>,
 <p><span class="pbl"><strong>판매가 : </strong></span><span class="pbr"><strong>17,820</strong>원<span>(10% off)</span></span></p>,
 <p><span class="pbl">마일리지 : </span><span class="pbr">990점 (5%)</span></p>]

In [17]:
# 정가 
sub_lis2 = sub_soup.select_one('.payment_box.curr').find_all('p')
sub_lis2

[<p><span class="pbl">정가 : </span><span class="pbr"><del>19,800원</del></span></p>,
 <p><span class="pbl"><strong>판매가 : </strong></span><span class="pbr"><strong>17,820</strong>원<span>(10% off)</span></span></p>,
 <p><span class="pbl">마일리지 : </span><span class="pbr">990점 (5%)</span></p>]

In [18]:
# 정가
ori_price = sub_lis2[0].get_text().split(':')[1]
ori_price

' 19,800원'

In [19]:
import re
re.sub('[^0-9]','',ori_price)

'19800'

In [20]:
# 판가
price = sub_lis2[1].get_text().split(':')[1]
price = price.split('원')[0]
price = int(price.replace(',',''))
price

17820

In [21]:
# 할인율 
off = sub_lis2[1].get_text().split(':')[1]
off = off.split('원')[1][1:-1]
off


'10% off'

In [22]:
# 이걸 어떻게 처리하면 좋을까... 

In [23]:
# 마일리지 
mile = sub_lis2[2].get_text().split(':')[1]
mile = int(mile.split('점')[0])
mile


990

In [24]:
# 적립율
pct = sub_lis2[2].get_text().split(':')[1]
pct.split('점')[1][2:-1]


'5%'

### 페이지에 있는 전체 데이터 가져오기

- 1페이지

In [68]:
lines = []
for li in lis : 
    title = li.select('.info > p')[0].get_text().strip()
    author = li.select('.info > p')[1].get_text().strip()
    href = li.find('a')['href']     # href 지정!! 

    sub_url = site + href 
    sub_req = requests.get(sub_url)
    sub_html = sub_req.text
    sub_soup = BeautifulSoup(sub_html, 'html.parser')
    sub_lis = sub_soup.select('.info_list > li')
    sub_lis2 = sub_soup.select_one('.payment_box.curr').find_all('p')


    if sub_list[:2] == '번역':
        trans = sub_soup.select('.info_list > li')[1].get_text()[5:]
        date = sub_lis[2].get_text().split(':')[1]
        # date = int(date.replace('-',''))
        page = sub_lis[3].get_text().split(':')[1]
        # page = int(page.replace('쪽',''))
    else: 
        trans = print('.')
        date = sub_lis[1].get_text().split(':')[1]
        # date = int(date.replace('-',''))
        page = sub_lis[2].get_text().split(':')[1]
        # page = int(page.replace('쪽',''))


    

    ori_price = sub_lis2[0].get_text().split(':')[1]
    ori_price = re.sub('[^0-9]','',ori_price)

    price = sub_lis2[1].get_text().split(':')[1]
    price = price.split('원')[0]
    price = int(price.replace(',',''))

    off = sub_lis2[1].get_text().split(':')[1]
    off = off.split('원')[1][1:-1]

    mile = sub_lis2[2].get_text().split(':')[1]
    # mile = int(mile.split('점')[0])
    
    pct = sub_lis2[2].get_text().split(':')[1]
    pct = pct.split('점')[1][2:-1]




    lines.append([title, author, trans, date, page, ori_price, price, off, mile, pct])
    
df = pd.DataFrame(lines, columns = ['제목', '저자', '역자', '출간일', '페이지', '정가', '판가','할인율','마일리지', '적립율'])
df.head()

   
    
    
    


.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.


,제목,저자,역자,출간일,페이지,정가,판가,할인율,마일리지,적립율
0,하루 한 장 초등과학 365,치바 가즈요시,None,"허영은 , 유우종 (감수)",2021-12-14,19800,17820,10% off,990점 (5%),5%
1,"다시, 배우다",폴 김,None,2021-12-10,304 쪽,16000,14400,10% off,800점 (5%),5%
2,당당한 디자인 결정을 위한 9가지 방법,톰 그리버,None,"김민성 , 곽서희",2021-12-10,22000,19800,10% off,"1,100점 (5%)",5%
3,365 부모 말하기 연습 일력(스프링북),"박재연 , 공인영",None,2021-12-01,384 쪽,18000,16200,10% off,900점 (5%),5%
4,공학도를 위한 수치해석(8판),"Steven C.Chapra , Raymond P. Canale",None,"나양 , 이승배 , 임윤철",2021-11-30,36000,36000,0% off,"1,080점 (3%)",3%
